In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import math
from sklearn.metrics import r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM,Dropout
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

plt.rcParams['xtick.labelsize'] = 24
plt.rcParams['ytick.labelsize'] = 24
plt.rcParams.update({'font.size':20})
plt.rcParams["figure.figsize"] = (12,10)

In [2]:
str_files = ''
#Carrega os dados do arquivo .csv
ugrhi06 = pd.read_csv(str_files + "cetesb_concatenado06semoutliers.csv",encoding='utf-8',sep=',')

#Carrega os dados do arquivo .csv
ugrhi10 = pd.read_csv(str_files + "cetesb_concatenado10semoutliers.csv",encoding='utf-8',sep=',')

ugrhi10

,solido,ph,coliformes,dbo,fosforo,od,temperatura,turbidez
0,189.791667,7.094057,3535.833333,4.916667,1.084160,2.471667,21.791667,27.641667
1,192.118607,7.105961,3439.495434,4.788813,1.084160,2.442676,21.951484,27.898333
2,194.694863,7.105961,3332.835616,4.647260,1.084160,2.410579,22.128425,28.182500
3,197.188014,7.120679,3229.616438,4.510274,1.084160,2.379517,22.299658,28.457500
4,199.764269,7.120679,3122.956621,4.368721,1.084160,2.347420,22.476598,28.741667
...,...,...,...,...,...,...,...,...
497,361.506122,7.136287,5700.000000,7.481633,0.197539,3.799184,24.995918,28.922449
498,345.626531,7.136287,5700.000000,8.620408,0.195135,3.849796,25.248980,33.730612
499,329.746939,7.206898,5700.000000,9.759184,0.192731,3.900408,25.502041,38.538776
500,314.379592,7.134653,5700.000000,10.861224,0.190404,3.949388,25.746939,43.191837


In [3]:
print('---- Tamanho dos conjuntos originais ----')
print('X:',len(ugrhi06))
print('y:',len(ugrhi10))

#Seleciona os dados equivalente ao tamanho da URGHI10
ugrhi06 = ugrhi06.iloc[0:len(ugrhi10)]

print('---- Conjunto de dados y igualado ao tamanho de dados do conjunto X ----')
print('X:',len(ugrhi06))
print('y:',len(ugrhi10))

# OBS.: O parâmetro temperatura não possui correlação suficiente para este tipo de experimento.
def seleciona_colunas(trainingd,p):
   
    posicao_coluna = 0
    frame_completo = pd.DataFrame()
   
    if (p == 'ph'):
        fcoliformes = trainingd.iloc[:,1:2].values
        fdbo     = trainingd.iloc[:,2:3].values
        fph      = trainingd.iloc[:,0:1].values
        fsolido  = trainingd.iloc[:,5:6].values
       
        frame_completo = pd.DataFrame(list(zip(fcoliformes,fdbo,fsolido,fph)),columns =['coliformes','dbo','solido','ph'])
        posicao_coluna = 3
       
    elif (p == 'od'):
        
        fod      = trainingd.iloc[:,4:5].values
        fsolido  = trainingd.iloc[:,5:6].values
   
        frame_completo = pd.DataFrame(list(zip(fsolido,fod)),columns =['solido','od'])
        posicao_coluna = 1
       
    elif (p == 'coliformes'):
        fph          = trainingd.iloc[:,0:1].values
        fcoliformes  = trainingd.iloc[:,1:2].values
     
        frame_completo = pd.DataFrame(list(zip(fph,fcoliformes)),columns =['ph','coliformes'])
        posicao_coluna = 1
       
    elif (p == 'dbo'):
        #ph, fosforo, solido tem correlação com o dbo
        fph      = trainingd.iloc[:,0:1].values
        fdbo     = trainingd.iloc[:,2:3].values
        ffosforo = trainingd.iloc[:,3:4].values
        fsolido  = trainingd.iloc[:,5:6].values
     
        frame_completo = pd.DataFrame(list(zip(fph,ffosforo,fsolido,fdbo)),columns =['ph','fosforo','solido','dbo'])
        posicao_coluna = 3
       
    elif (p == 'fosforo'):
        # dbo e solido tem correlação com o fosforo
        fdbo     = trainingd.iloc[:,2:3].values
        ffosforo = trainingd.iloc[:,3:4].values
        fsolido  = trainingd.iloc[:,5:6].values
     
        frame_completo = pd.DataFrame(list(zip(fdbo,fsolido,ffosforo)),columns =['dbo','solido','fosforo'])
        posicao_coluna = 2
       
    elif (p == 'solido'):
        # ph, dbo, fosforo, od, solido e turbidez tem correlação com o solido
        fph        = trainingd.iloc[:,0:1].values
        fdbo       = trainingd.iloc[:,2:3].values
        ffosforo   = trainingd.iloc[:,3:4].values
        fod        = trainingd.iloc[:,4:5].values
        fsolido    = trainingd.iloc[:,5:6].values
        fturbidez  = trainingd.iloc[:,7:8].values
     
        frame_completo = pd.DataFrame(list(zip(fph,fdbo,ffosforo,fod,fturbidez,fsolido)),
                                  columns =['ph','dbo','fosforo','od','turbidez','solido'])
        posicao_coluna = 5
       
    elif (p == 'turbidez'):
        # solido e turbidez tem correlação com o turbidez
   
        fsolido    = trainingd.iloc[:,5:6].values
        fturbidez  = trainingd.iloc[:,7:8].values
     
        frame_completo = pd.DataFrame(list(zip(fsolido,fturbidez)),columns =['solido','turbidez'])
        posicao_coluna = 1
       
    return frame_completo,posicao_coluna


def prepara_x_y_train_previsao(df6,df10,p,index_coluna):
    
    #Seleciona os dados, menos a coluna predita.
    X = df6.iloc[:,0:index_coluna].values
    
    #Seleciona os valores da coluna ph da URGHI06, na qual será utilizado para a normalização dos dados quando apresentar o gráfico
    train_previsao = df6[p].values
    #Seleciona os valores da coluna ph da URGHI10.
    y = df10[p].values
        
    X = X.reshape(len(X),index_coluna)
    y = y.reshape(len(y),1)
    train_previsao = train_previsao.reshape(len(train_previsao),1)
        
    return X,y,train_previsao

'''

def seleciona_colunas(trainingd,p):
    
    if (p == 'ph'):
        fdbo     = trainingd.iloc[:,2:3].values
        fph      = trainingd.iloc[:,0:1].values
        fsolido  = trainingd.iloc[:,5:6].values
   
        frame_completo = pd.DataFrame(list(zip(fdbo,fsolido,fph)),columns =['dbo','solido','ph'])
    else:
        
        fdbo     = trainingd.iloc[:,2:3].values
        fod      = trainingd.iloc[:,4:5].values
        fsolido  = trainingd.iloc[:,5:6].values
   
        frame_completo = pd.DataFrame(list(zip(fdbo,fsolido,fod)),columns =['dbo','solido','od'])
    
        
    return frame_completo

df_ugrhi06 = seleciona_colunas(ugrhi06,'od')

#Seleciona os dados equivalente ao tamanho da URGHI10
X = df_ugrhi06.iloc[0:len(ugrhi10)]
#Remove a coluna ph, pois os seus valores serão preditos na UGRHI10
X = X.drop(['od'],axis = 1).values
#Seleciona os valores da coluna ph da URGHI06, na qual será utilizado para a normalização dos dados quando apresentar o gráfico
train_previsao = ugrhi06['od'].values
#Seleciona os valores da coluna ph da URGHI10.
y = ugrhi10['od'].values

X = X.reshape(len(X),2)
y = y.reshape(len(y),1)
train_previsao = train_previsao.reshape(len(train_previsao),1)


assert X.shape == (len(X),2)
assert y.shape == (len(y),1)

print('---- Conjunto de dados y igualado ao tamanho de dados do conjunto X ----')
print('X:',len(X))
print('y:',len(y))
'''

---- Tamanho dos conjuntos originais ----
X: 503
y: 502
---- Conjunto de dados y igualado ao tamanho de dados do conjunto X ----
X: 502
y: 502


"\n\ndef seleciona_colunas(trainingd,p):\n    \n    if (p == 'ph'):\n        fdbo     = trainingd.iloc[:,2:3].values\n        fph      = trainingd.iloc[:,0:1].values\n        fsolido  = trainingd.iloc[:,5:6].values\n   \n        frame_completo = pd.DataFrame(list(zip(fdbo,fsolido,fph)),columns =['dbo','solido','ph'])\n    else:\n        \n        fdbo     = trainingd.iloc[:,2:3].values\n        fod      = trainingd.iloc[:,4:5].values\n        fsolido  = trainingd.iloc[:,5:6].values\n   \n        frame_completo = pd.DataFrame(list(zip(fdbo,fsolido,fod)),columns =['dbo','solido','od'])\n    \n        \n    return frame_completo\n\ndf_ugrhi06 = seleciona_colunas(ugrhi06,'od')\n\n#Seleciona os dados equivalente ao tamanho da URGHI10\nX = df_ugrhi06.iloc[0:len(ugrhi10)]\n#Remove a coluna ph, pois os seus valores serão preditos na UGRHI10\nX = X.drop(['od'],axis = 1).values\n#Seleciona os valores da coluna ph da URGHI06, na qual será utilizado para a normalização dos dados quando apresentar 

In [4]:
#Funções para cálculos das métricas de desempenho e ajuste da estrutura dos dados.
'''
def mape(y_pred,y_true):
    mape_sum = 0
    for real,prediction in zip(y_true,y_pred):
        mape_sum += (abs((real - prediction))/real)
        mape = mape_sum/len(real)
    return mape
'''

def mape(y_pred,y_true):
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    return mape

def rmse(y_pred,y_true):
    mse = mean_squared_error(y_true, y_pred)
    rmse = math.sqrt(mse)
    return rmse

def r2(y_pred,y_true):
    rscore = r2_score(y_true,y_pred)
    return rscore

#Função para calcular o coeficiente de determinação entre os valores observados e preditos.
#Calcula-se a correlação entre os valores observados e preditos e eleva ao expoente 2.
def correlacao_determinacao(dtframe,tipo):
    
    if (tipo == 0): #Treino
        resultado = dtframe.corr().previsao_treino.values[1] ** 2 
        
    else: #teste
        resultado = dtframe.corr().previsao_teste.values[1] ** 2
            
    return resultado

def pegar_dados_coluna_test(trainingd,percent,index_coluna):
    data = trainingd.iloc[:,:].values
    train_previsao = trainingd.iloc[0:int(len(data)*percent),index_coluna:index_coluna+1].values
           
    return train_previsao

def ajusta_lista(array):
    lista = []
    
    for i in range(len(array)):
        lista.append(array[i][0])
            
    return lista

#Monta o Dataframe para calcular a correlação e coeficiente de determinação entre os valores observados e preditos.
def df_correlacao(previsoes_treino,treino,previsoes,observado_test):
    
    #print(ajusta_lista(treino))
    
    df_corr_determinacao_treino = pd.DataFrame()
    df_corr_determinacao_teste  = pd.DataFrame()
      
    #Calcula o coeficiente de determinação
    
    dict = {'previsao_treino': ajusta_lista(previsoes_treino), 'treino': ajusta_lista(treino)} 
        
    df_treino = pd.DataFrame(dict)
    dframes_treino = [df_corr_determinacao_treino,df_treino]
    df_corr_determinacao_treino = pd.concat(dframes_treino)
    
    dict = {'previsao_teste': ajusta_lista(previsoes), 'teste': ajusta_lista(observado_test)} 
    
    df_teste = pd.DataFrame(dict)
    dframes_teste = [df_corr_determinacao_teste,df_teste]
    df_corr_determinacao_teste = pd.concat(dframes_teste)
    
    r_treino = round(correlacao_determinacao(df_corr_determinacao_treino,0),4)
    r_teste  = round(correlacao_determinacao(df_corr_determinacao_teste,1),4)
                
    return r_treino,r_teste

In [5]:
df_resultado_medio_g = pd.DataFrame()

#index_coluna = 0

parametro = []

#média
media_parametro_mape_treino = []
media_parametro_rmse_treino = []
media_parametro_r_treino    = []
   
media_parametro_mape_teste  = []
media_parametro_rmse_teste  = []
media_parametro_r_teste     = []
   
#desvio_padrão
   
#std_parametro_mape_treino = []
std_parametro_rmse_treino = []
#std_parametro_r_treino    = []
   
#std_parametro_mape_teste  = []
std_parametro_rmse_teste  = []
#std_parametro_r_teste     = []

tecnica = []

for p in ('ph','coliformes', 'dbo','fosforo','od','solido','turbidez'):
   
    print('Agora é a vez do parâmetro:', p)
    
    df_ugrhi06,index_coluna = seleciona_colunas(ugrhi06,p)

    X,y,train_previsao = prepara_x_y_train_previsao(df_ugrhi06,ugrhi10,p,index_coluna)

    #Normalização dos dados: Normaliza os dados dentro um intervalo (0 a 1).
    sc = MinMaxScaler()
    X = sc.fit_transform(X)
    y = sc.fit_transform(y)

    #Normalização dos dados de previsão para teste
    normalizador_previsao = MinMaxScaler()
    normalizador_previsao.fit_transform(train_previsao)

    #Separação dos dados para treinamento(70%) e teste(30%)
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3)

    #am = ['rl','rf','mlp','lstm']
    
    media_simulador_mape_treino = []
    media_simulador_rmse_treino = []
    media_simulador_r_treino    = []
            
    media_simulador_mape_teste  = []
    media_simulador_rmse_teste  = []
    media_simulador_r_teste     = []

    for t in ('rl','rf','mlp','lstm'):
        if t == 'rl':
            #print('REGRESSÃO LINEAR')
            model = LinearRegression()
        
            #Treina o modelo
            model.fit(X_train,y_train)
        
            #Predição dos dados de teste
            y_pred = model.predict(X_test)

            y_pred_rl = sc.inverse_transform(y_pred)

            X_test_rl = sc.inverse_transform(X_test)
            y_test_rl = sc.inverse_transform(y_test)
        
            #Predição dos dados de treinamento
            y_predt = model.predict(X_train)
            y_predt_rl = sc.inverse_transform(y_predt)
        
            y_train_rl = sc.inverse_transform(y_train)
        
            '''
            plt.plot(y_test_rl,color='red', label='pH Observado')
            plt.plot(y_pred_rl,color='blue', label='pH predito')
            plt.xlabel('Semanas')
            plt.ylabel('Valor de pH')
            plt.legend()
            plt.title('Predição Regressão Linear')
            plt.show()
            '''
        
            mape_treino_rl       = round(mape(y_predt_rl,y_train_rl),2)
            rmse_treino_rl       = round(rmse(y_predt_rl,y_train_rl),2)
                
            mape_teste_rl        = round(mape(y_pred_rl,y_test_rl),2)
            rmse_teste_rl        = round(rmse(y_pred_rl,y_test_rl),2)
                
            r_treino_rl,r_teste_rl = df_correlacao(y_predt_rl,y_train_rl,y_pred_rl,y_test_rl)
            
            media_simulador_mape_treino.append(np.mean(mape_treino_rl))
            media_simulador_rmse_treino.append(np.mean(rmse_treino_rl))
            media_simulador_r_treino.append(np.mean(r_treino_rl))
            
            media_simulador_mape_teste.append(np.mean(mape_teste_rl))
            media_simulador_rmse_teste.append(np.mean(rmse_teste_rl))
            media_simulador_r_teste.append(np.mean(r_teste_rl))
            
            tecnica.append('RL (Var. selec.)')
            parametro.append(p)
                        
        elif t == 'rf':
            #print('RANDON FOREST')
        
            model = RandomForestRegressor()
        
            #Treina o modelo
            model.fit(X_train,y_train)

            #Predição dados de teste
            y_pred = model.predict(X_test)
                
            y_pred_rf = sc.inverse_transform(y_pred.reshape(-1, 1))
        
            X_test_rf = sc.inverse_transform(X_test)
            y_test_rf = sc.inverse_transform(y_test)
        
            #Predição dados de treinamento
        
            y_predt = model.predict(X_train)
            y_predt_rf = sc.inverse_transform(y_predt.reshape(-1, 1))
        
            y_train_rf = sc.inverse_transform(y_train)
        
            '''
            plt.plot(y_test_rf,color='red', label='pH Observado')
            plt.plot(y_pred_rf,color='blue', label='pH predito')
            plt.xlabel('Semanas')
            plt.ylabel('Valor de pH')
            plt.legend()
            plt.title('Predição Random Forest')
            plt.show()
            '''
        
            mape_treino_rf       = round(mape(y_predt_rf,y_train_rf),2)
            rmse_treino_rf       = round(rmse(y_predt_rf,y_train_rf),2)
        
            mape_teste_rf        = round(mape(y_pred_rf,y_test_rf),2)
            rmse_teste_rf        = round(rmse(y_pred_rf,y_test_rf),2)
                
            r_treino_rf,r_teste_rf = df_correlacao(y_predt_rf,y_train_rf,y_pred_rf,y_test_rf)
            
            media_simulador_mape_treino.append(np.mean(mape_treino_rf))
            media_simulador_rmse_treino.append(np.mean(rmse_treino_rf))
            media_simulador_r_treino.append(np.mean(r_treino_rf))
            
            media_simulador_mape_teste.append(np.mean(mape_teste_rf))
            media_simulador_rmse_teste.append(np.mean(rmse_teste_rf))
            media_simulador_r_teste.append(np.mean(r_teste_rf))
            
            tecnica.append('RF (Var. selec.)')
            parametro.append(p)
        
        
        elif t == 'mlp':
        
            #print('MLP')           
            
        
            for r in range(0,6):
                               
                model = Sequential()
                model.add(Dense(units = 10, activation = 'relu'))
                model.add(Dense(units = 21, activation = 'relu'))
                model.add(Dense(units = 1, activation = 'sigmoid'))
                model.compile(loss = 'mean_absolute_error', optimizer = 'adam',metrics = ['mean_absolute_error'])
        
                es = EarlyStopping(monitor='val_loss', patience = 3, verbose=0)


                #Treina o modelo
                history = model.fit(X_train, y_train, validation_data=(X_test,y_test), batch_size = 32, epochs = 2000, 
                        callbacks=[es], verbose=0)

                #Predição com os dados de test
                y_pred = model.predict(X_test)
                y_pred_mlp = normalizador_previsao.inverse_transform(y_pred)

                X_test_mlp = sc.inverse_transform(X_test)
                y_test_mlp = sc.inverse_transform(y_test)
        
                #Predição com os dados de treinamento
                y_predt = model.predict(X_train)
                y_predt_mlp = normalizador_previsao.inverse_transform(y_predt)
        
                y_train_mlp = sc.inverse_transform(y_train)
        
                '''
                plt.plot(y_test_mlp,color='red', label='pH Observado')
                plt.plot(y_pred_mlp,color='blue', label='pH predito')
                plt.xlabel('Semanas')
                plt.ylabel('Valor de pH')
                plt.legend()
                plt.title('Predição MLP')
                plt.show()
                '''
        
                mape_treino_mlp_sigmoid       = round(mape(y_predt_mlp,y_train_mlp),2)
                rmse_treino_mlp_sigmoid       = round(rmse(y_predt_mlp,y_train_mlp),2)
        
                mape_teste_mlp_sigmoid        = round(mape(y_pred_mlp,y_test_mlp),2)
                rmse_teste_mlp_sigmoid        = round(rmse(y_pred_mlp,y_test_mlp),2)
        
                r_treino_mlp_sigmoid,r_teste_mlp_sigmoid = df_correlacao(y_predt_mlp,y_train_mlp,y_pred_mlp,y_test_mlp)
               
                media_simulador_mape_treino.append(np.mean(mape_treino_mlp_sigmoid))
                media_simulador_rmse_treino.append(np.mean(rmse_treino_mlp_sigmoid))
                media_simulador_r_treino.append(np.mean(r_treino_mlp_sigmoid))
            
                media_simulador_mape_teste.append(np.mean(mape_teste_mlp_sigmoid))
                media_simulador_rmse_teste.append(np.mean(rmse_teste_mlp_sigmoid))
                media_simulador_r_teste.append(np.mean(r_teste_mlp_sigmoid))
                
                
                '''
                # plot training history
                plt.plot(history.history['loss'], label='train')
                plt.plot(history.history['val_loss'], label='test')
                plt.legend()
                plt.show()
                '''
                
            tecnica.append('MLP (Var. selec.)')
            parametro.append(p)
                   
    
        else:
        
                #print('LSTM')
                    
            X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
            X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
                
                                
            for r in range(0,6):
                
                model = Sequential()
                model.add(LSTM(units = 10, input_shape = (X_train.shape[1], 1)))
                model.add(Dense(21, activation = 'relu'))
                model.add(Dropout(0.2))
                model.add(Dense(1, activation = 'sigmoid'))
                model.compile(loss = 'mean_absolute_error', optimizer = 'adam',metrics = ['mean_absolute_error'])

                es = EarlyStopping(monitor='val_loss', patience = 3, verbose=0)


                #Treina o modelo
                history = model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size = 32, epochs = 2000, 
                        callbacks=[es], verbose=0)

                #Predição com dados de teste
                y_pred = model.predict(X_test)
                y_pred_lstm = sc.inverse_transform(y_pred)

                y_test_lstm = sc.inverse_transform(y_test)
                y_test_lstm.shape
        
                #Predição com dados de treinamento
                y_predt = model.predict(X_train)
                y_predt_lstm = sc.inverse_transform(y_predt)

                y_train_lstm = sc.inverse_transform(y_train)
        
                '''
                plt.plot(y_test_lstm,color='red', label='pH Observado')
                plt.plot(y_pred_lstm,color='blue', label='pH predito')
                plt.xlabel('Semanas')
                plt.ylabel('Valor de pH')
                plt.legend()
                plt.title('Predição LSTM')
                plt.show()
        
                '''
        
                mape_treino_lstm_sigmoid       = round(mape(y_predt_lstm,y_train_lstm),2)
                rmse_treino_lstm_sigmoid       = round(rmse(y_predt_lstm,y_train_lstm),2)
                
                mape_teste_lstm_sigmoid        = round(mape(y_pred_lstm,y_test_lstm),2)
                rmse_teste_lstm_sigmoid        = round(rmse(y_pred_lstm,y_test_lstm),2)
                
                r_treino_lstm_sigmoid,r_teste_lstm_sigmoid = df_correlacao(y_predt_lstm,y_train_lstm,y_pred_lstm,y_test_lstm)
                
                '''
                # plot training history
                plt.plot(history.history['loss'], label='train')
                plt.plot(history.history['val_loss'], label='test')
                plt.legend()
                plt.show()
                '''
                    
                media_simulador_mape_treino.append(np.mean(mape_treino_lstm_sigmoid))
                media_simulador_rmse_treino.append(np.mean(rmse_treino_lstm_sigmoid))
                media_simulador_r_treino.append(np.mean(r_treino_lstm_sigmoid))
            
                media_simulador_mape_teste.append(np.mean(mape_teste_lstm_sigmoid))
                media_simulador_rmse_teste.append(np.mean(rmse_teste_lstm_sigmoid))
                media_simulador_r_teste.append(np.mean(r_teste_lstm_sigmoid))
                
                                    
            tecnica.append('LSTM (Var. selec.)')
            parametro.append(p)
                    
                #média
        media_parametro_mape_treino.append(np.mean(media_simulador_mape_treino))
        media_parametro_rmse_treino.append(np.mean(media_simulador_rmse_treino))
        media_parametro_r_treino.append(np.mean(media_simulador_r_treino))
    
        media_parametro_mape_teste.append(np.mean(media_simulador_mape_teste))
        media_parametro_rmse_teste.append(np.mean(media_simulador_rmse_teste))
        media_parametro_r_teste.append(np.mean(media_simulador_r_teste))
        
        #desvio_padrão
    
        #std_parametro_mape_treino.append(np.std(media_lag_mape_treino))
        std_parametro_rmse_treino.append(np.std(media_simulador_rmse_treino))
        #std_parametro_r_treino.append(np.std(media_lag_r_treino))
    
        #std_parametro_mape_teste.append(np.std(media_lag_mape_teste))
        std_parametro_rmse_teste.append(np.std(media_simulador_rmse_teste))
        #std_parametro_r_teste.append(np.std(media_lag_r_teste))
'''
print(len(parametro))
print(len(tecnica))
print(len(media_parametro_mape_treino))
print(len(media_parametro_rmse_treino))
print(len(std_parametro_rmse_treino))
print(len(media_parametro_r_treino))
print(len(media_parametro_mape_teste))
print(len(media_parametro_rmse_teste))
print(len(std_parametro_rmse_teste))
print(len(media_parametro_r_teste))
'''    
    
dict = {'parametro': parametro,
        'AM':tecnica,
        'mape_treino':  media_parametro_mape_treino,
        'rmse_treino':  media_parametro_rmse_treino, 
        'std_rmse_treino':  std_parametro_rmse_treino,
        'r_quad_treino':media_parametro_r_treino,    
        'mape_teste':   media_parametro_mape_teste,   
        'rmse_teste':   media_parametro_rmse_teste,  
        'std_rmse_teste':   std_parametro_rmse_teste,
        'r_quad_teste': media_parametro_r_teste}
   
df_resultado_final = pd.DataFrame(dict)

dframes = [df_resultado_medio_g,df_resultado_final]
df_resultado_medio_g = pd.concat(dframes)
df_resultado_medio_g.round(2)


Agora é a vez do parâmetro: ph


/home/anderson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Agora é a vez do parâmetro: coliformes


/home/anderson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Agora é a vez do parâmetro: dbo


...

Agora é a vez do parâmetro: solido


/home/anderson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Agora é a vez do parâmetro: turbidez


/home/anderson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


,parametro,AM,mape_treino,rmse_treino,std_rmse_treino,r_quad_treino,mape_teste,rmse_teste,std_rmse_teste,r_quad_teste
0,ph,RL (Var. selec.),0.37,0.03,0.00,0.08,0.35,0.03,0.00,0.09
1,ph,RF (Var. selec.),0.26,0.02,0.01,0.50,0.36,0.03,0.00,0.07
2,ph,MLP (Var. selec.),3.66,0.27,0.14,0.17,3.77,0.28,0.14,0.07
3,ph,LSTM (Var. selec.),2.25,0.17,0.16,0.13,2.31,0.17,0.16,0.08
4,coliformes,RL (Var. selec.),49365.03,30072.59,0.00,0.04,80059.58,30214.37,0.00,0.02
5,coliformes,RF (Var. selec.),30217.74,24273.08,5799.50,0.35,89179.26,32850.39,2636.02,0.01
6,coliformes,MLP (Var. selec.),912788.16,576927.51,319297.34,0.12,1501994.50,591440.84,322737.59,0.01
7,coliformes,LSTM (Var. selec.),533923.92,343632.05,361699.66,0.08,878310.66,351762.64,368936.24,0.01
8,dbo,RL (Var. selec.),63.98,7.45,0.00,0.10,81.46,7.12,0.00,0.18
9,dbo,RF (Var. selec.),42.95,5.19,2.26,0.51,81.32,7.16,0.04,0.17


In [55]:
print('ESPACIAL VAR. SELECIONADAS')
df_resultado_medio_g.round(2).to_csv('/home/anderson/Downloads/espacial_multivariaveis_e_selecionadas/espacial_var_selec.csv',index=True,header=True)

ESPACIAL VAR. SELECIONADAS


In [14]:
df_resultados_teste

,AM,MAPE,RMSE,R²,TIPO
0,Regressão Linear,2.7211,0.2481,0.0003,TESTE
1,Random Forest,3.1977,0.2824,0.0031,TESTE
2,MLP,5.5112,0.4600,0.0039,TESTE
3,LSTM,2.7118,0.2459,0.0014,TESTE
